# 1️⃣ Setup: 환경 설정 및 프로젝트 준비

이 노트북은 다음을 수행합니다:
1. 경로 설정 (사용자 환경에 맞게 수정)
2. 가상환경 연결 및 확인
3. Git Repository 클론
4. 데이터셋 다운로드 (DIV2K, 벤치마크)
5. CATANet-L x2 원본 모델 PTL 변환

---
## ⚙️ 0. 경로 설정 (이 셀만 수정하세요!)

**사용 환경에 맞게 아래 경로만 수정하면 됩니다.**

In [1]:
# ============================================================
# 🔧 사용자 설정 - 환경에 맞게 수정하세요!
# ============================================================

# 기본 경로 (home 디렉토리 기준)
BASE_DIR = "my_ssl"  # home 아래 작업 디렉토리

# 가상환경 이름
VENV_NAME = "ssl_env"

# Python 버전 (site-packages 경로용)
PYTHON_VERSION = "3.12"

# Git 설정
GIT_REPO_URL = "https://github.com/paaran2024/uos-ssl-project.git"
GIT_BRANCH = "ai"
REPO_NAME = "uos-ssl-project"

# ============================================================
# 🔒 아래는 자동 설정 (수정 불필요)
# ============================================================
import os
os.environ['MPLBACKEND'] = 'Agg'  # matplotlib 서버 호환

home_dir = os.path.expanduser('~')
base_path = os.path.join(home_dir, BASE_DIR)

# 가상환경 경로
venv_path = os.path.join(base_path, VENV_NAME)
site_packages = os.path.join(venv_path, 'lib', f'python{PYTHON_VERSION}', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')
pip_exec = os.path.join(venv_path, 'bin', 'pip')

# 프로젝트 경로
project_dir = os.path.join(base_path, REPO_NAME)
ai_dir = os.path.join(project_dir, 'ai')

print("="*60)
print("📁 경로 설정 완료")
print("="*60)
print(f"Home:        {home_dir}")
print(f"Base:        {base_path}")
print(f"Venv:        {venv_path}")
print(f"Project:     {project_dir}")
print(f"AI Dir:      {ai_dir}")
print(f"Python:      {python_exec}")
print("="*60)

📁 경로 설정 완료
Home:        /home1/uos0808
Base:        /home1/uos0808/my_ssl
Venv:        /home1/uos0808/my_ssl/ssl_env
Project:     /home1/uos0808/my_ssl/uos-ssl-project
AI Dir:      /home1/uos0808/my_ssl/uos-ssl-project/ai
Python:      /home1/uos0808/my_ssl/ssl_env/bin/python


## 1.1 가상환경 설정

In [2]:
import sys

# sys.path에 가상환경 추가
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)
    print(f"✅ 가상환경 라이브러리 경로 추가됨")

# 가상환경 존재 여부 확인
if not os.path.exists(venv_path):
    print(f"🚀 가상환경 생성 중: {venv_path}")
    !python3 -m venv {venv_path}
    !{pip_exec} install --upgrade pip -q
    print("✅ 가상환경 생성 완료")
else:
    print(f"✅ 가상환경 확인됨: {venv_path}")

✅ 가상환경 라이브러리 경로 추가됨
✅ 가상환경 확인됨: /home1/uos0808/my_ssl/ssl_env


## 1.2 GPU 확인

In [3]:
import torch
print(f"✅ PyTorch Version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"✅ CUDA Available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ CUDA Not Available")

✅ PyTorch Version: 2.9.1+cu128
✅ CUDA Available: NVIDIA RTX A6000
   Memory: 50.9 GB


## 1.3 Git Repository 클론

In [4]:
if os.path.exists(project_dir):
    print("✅ 폴더가 이미 존재합니다. 최신 코드로 업데이트합니다.")
    %cd {project_dir}
    !git fetch origin {GIT_BRANCH}
    !git pull origin {GIT_BRANCH}
else:
    print(f"🚀 Git 클론 중: {GIT_REPO_URL}")
    os.makedirs(base_path, exist_ok=True)
    %cd {base_path}
    !git clone -b {GIT_BRANCH} {GIT_REPO_URL} {REPO_NAME}
    print("✅ 클론 완료")

%cd {ai_dir}
print(f"\n📁 현재 디렉토리: {os.getcwd()}")

✅ 폴더가 이미 존재합니다. 최신 코드로 업데이트합니다.
/home1/uos0808/my_ssl/uos-ssl-project


/myubai/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


From https://github.com/paaran2024/uos-ssl-project
 * branch            ai         -> FETCH_HEAD
From https://github.com/paaran2024/uos-ssl-project
 * branch            ai         -> FETCH_HEAD
Already up to date.
/home1/uos0808/my_ssl/uos-ssl-project/ai

📁 현재 디렉토리: /home1/uos0808/my_ssl/uos-ssl-project/ai


## 1.4 필수 패키지 설치

In [5]:
print("📦 필수 패키지 설치 중...")
!{pip_exec} install torch einops pandas numpy pyyaml tqdm opencv-python-headless scikit-image matplotlib scipy -q
print("✅ 패키지 설치 완료")

📦 필수 패키지 설치 중...
✅ 패키지 설치 완료


## 1.5 DIV2K 데이터셋 다운로드

In [6]:
div2k_dir = os.path.join(ai_dir, 'datasets/DIV2K')
os.makedirs(div2k_dir, exist_ok=True)

%cd {div2k_dir}

# HR 이미지
if not os.path.exists('DIV2K_train_HR'):
    print("📥 DIV2K HR 다운로드 중...")
    !wget -q http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
    !unzip -q DIV2K_train_HR.zip && rm DIV2K_train_HR.zip
    print("✅ HR 완료")
else:
    print("✅ DIV2K HR 이미 존재")

# LR 이미지 (x2)
if not os.path.exists('DIV2K_train_LR_bicubic'):
    print("📥 DIV2K LR (x2) 다운로드 중...")
    !wget -q http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
    !unzip -q DIV2K_train_LR_bicubic_X2.zip && rm DIV2K_train_LR_bicubic_X2.zip
    print("✅ LR 완료")
else:
    print("✅ DIV2K LR 이미 존재")

print(f"\n📁 DIV2K 구조:")
!ls -la

/home1/uos0808/my_ssl/uos-ssl-project/ai/datasets/DIV2K
✅ DIV2K HR 이미 존재
✅ DIV2K LR 이미 존재

📁 DIV2K 구조:
total 4351635
drwxr-xr-x 4 uos0808 users       4096 Dec  8 13:27 .
drwxr-xr-x 4 uos0808 users       4096 Dec 10 13:10 ..
drwxr-xr-x 2 uos0808 users      32768 Dec  8 13:26 DIV2K_train_HR
-rw-r--r-- 1 uos0808 users 3530603713 Feb 14  2017 DIV2K_train_HR.zip
drwxr-xr-x 3 uos0808 users       4096 Dec  8 13:27 DIV2K_train_LR_bicubic
-rw-r--r-- 1 uos0808 users  925390592 Feb 14  2017 DIV2K_train_LR_bicubic_X2.zip


## 1.6 벤치마크 데이터셋 다운로드

In [13]:
benchmark_dir = os.path.join(ai_dir, 'datasets/benchmark')
os.makedirs(benchmark_dir, exist_ok=True)
%cd {benchmark_dir}

if not os.path.exists('Set5'):
    print("📥 벤치마크 데이터셋 다운로드 중...")
    
    url = "https://github.com/Haejyn/benchmark-upload/releases/download/zip/TestDataSR.1.zip"
    !wget --no-check-certificate "{url}" -O benchmark.zip
    
    import zipfile
    with zipfile.ZipFile('benchmark.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    os.remove('benchmark.zip')
    
    if os.path.exists('benchmark'):
        !mv benchmark/* . && rm -rf benchmark
    print("✅ 벤치마크 다운로드 완료")
else:
    print("✅ 벤치마크 데이터셋 이미 존재")

print(f"\n📁 벤치마크 구조:")
!ls -la

/home1/uos0808/my_ssl/uos-ssl-project/ai/datasets/benchmark
📥 벤치마크 데이터셋 다운로드 중...
--2025-12-10 14:43:59--  https://github.com/Haejyn/benchmark-upload/releases/download/zip/TestDataSR.1.zip
20.200.245.247ub.com (github.com)... 
Connecting to github.com (github.com)|20.200.245.247|:443... connected.
302 Foundest sent, awaiting response... 
Location: https://release-assets.githubusercontent.com/github-production-release-asset/1113503093/99b5151c-05e3-413c-ada9-84f5c018db6c?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-10T06%3A31%3A17Z&rscd=attachment%3B+filename%3DTestDataSR.1.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-10T05%3A30%3A30Z&ske=2025-12-10T06%3A31%3A17Z&sks=b&skv=2018-11-09&sig=w75CHn51%2B5ysh%2BkJfSac62w1TC%2FBBgHMV2EMkMHof6M%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTM0O

---
## 1.7 CATANet-L x2 원본 모델 PTL 변환

원본 CATANet-L x2 모델을 4가지 입력 해상도에 대해 PTL 파일로 변환합니다.
- 320x320, 640x640, 1280x1280, 1920x1920

**중요**: `flutter_pytorch_lite`는 LibTorch 1.13을 사용하므로, PyTorch 1.13 환경에서 변환해야 합니다.

In [ ]:
%cd {ai_dir}

print("="*60)
print("🔄 CATANet-L x2 원본 모델 PTL 변환")
print("="*60)

# ============================================================
# PyTorch 1.13 가상환경 설정 (flutter_pytorch_lite 호환용)
# ============================================================
torch13_venv = os.path.join(base_path, "torch13_env")
torch13_python = os.path.join(torch13_venv, 'bin', 'python')
torch13_pip = os.path.join(torch13_venv, 'bin', 'pip')

if not os.path.exists(torch13_venv):
    print("\n🔧 PyTorch 1.13 가상환경 생성 중...")
    print("   (flutter_pytorch_lite는 LibTorch 1.13을 사용합니다)")
    !python3 -m venv {torch13_venv}
    !{torch13_pip} install --upgrade pip -q
    !{torch13_pip} install torch==1.13.0 einops -q
    print("✅ PyTorch 1.13 환경 생성 완료")
else:
    print(f"✅ PyTorch 1.13 환경 확인됨: {torch13_venv}")

# PyTorch 버전 확인
!{torch13_python} -c "import torch; print(f'   PyTorch Version: {torch.__version__}')"

# ============================================================
# PTL 변환 (PyTorch 1.13 사용)
# ============================================================
import gc

input_sizes = [320, 640, 1280, 1920]
weights_path = "weights/CATANet-L_x2.pth"

if not os.path.exists(weights_path):
    print(f"❌ 원본 가중치 파일이 없습니다: {weights_path}")
else:
    for i, size in enumerate(input_sizes, 1):
        print(f"\n[{i}/4] 입력 크기: {size}x{size}")
        print("-"*40)
        
        output_path = f"weights/CATANet-L_x2_{size}.ptl"
        
        if os.path.exists(output_path):
            print(f"✅ 이미 존재: {output_path}")
            continue
        
        # PyTorch 1.13 환경에서 변환 실행
        !{torch13_python} scripts/convert_to_ptl.py \
            --weights {weights_path} \
            --input_size {size} \
            --upscale 2 \
            --output {output_path}
        
        # 메모리 정리
        gc.collect()
        
        if os.path.exists(output_path):
            file_size = os.path.getsize(output_path) / (1024*1024)
            print(f"✅ 완료: {output_path} ({file_size:.1f} MB)")

print("\n" + "="*60)
print("📁 생성된 PTL 파일 (LibTorch 1.13 호환):")
!ls -lh weights/*.ptl 2>/dev/null || echo "PTL 파일 없음"

---
## 1.8 설정 완료 확인

In [ ]:
%cd {ai_dir}

print("="*60)
print("✅ Setup 완료!")
print("="*60)
print(f"\n📁 프로젝트 경로: {project_dir}")
print(f"📁 AI 디렉토리:   {ai_dir}")
print(f"🐍 Python 실행:   {python_exec}")

print(f"\n📂 weights 폴더:")
!ls -lh weights/ 2>/dev/null || echo "weights 폴더 없음"

print("\n" + "="*60)
print("💡 다른 노트북에서 아래 설정을 복사하세요:")
print("="*60)
print(f'''
# ⚙️ 경로 설정 (1_setup.ipynb의 0번 셀과 동일하게)
BASE_DIR = "{BASE_DIR}"
VENV_NAME = "{VENV_NAME}"
PYTHON_VERSION = "{PYTHON_VERSION}"
REPO_NAME = "{REPO_NAME}"

import os
os.environ['MPLBACKEND'] = 'Agg'
home_dir = os.path.expanduser('~')
base_path = os.path.join(home_dir, BASE_DIR)
venv_path = os.path.join(base_path, VENV_NAME)
site_packages = os.path.join(venv_path, 'lib', f'python{{PYTHON_VERSION}}', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')
project_dir = os.path.join(base_path, REPO_NAME)
ai_dir = os.path.join(project_dir, 'ai')
''')